# 🗺️Geospatial Analysis 
> __*On Bishkek Air Condition data*__

![Geospatial Analysis in Vrubel style](https://user-images.githubusercontent.com/115424463/282264815-cba13dc0-ac16-4bc6-8628-112876bd1364.jpeg)

> Picture: Geospatial Analysis in Vrubel style

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, FeatureGroup
from folium.plugins import MarkerCluster, MousePosition
import math

from warnings import simplefilter
simplefilter("ignore")

In [ ]:
sensors_df = pd.read_csv('/kaggle/input/2023-11-10-bishkek-air-condition/2023_11_10_Bishkek_Air.csv')
sensors = gpd.GeoDataFrame(sensors_df, geometry=gpd.points_from_xy(sensors_df.lon, sensors_df.lat))
sensors.crs = {'init': 'epsg:4326'}
sensors

In [ ]:
# Find Bishkek's geolocation
world_filepath = gpd.datasets.get_path('naturalearth_cities')
world = gpd.read_file(world_filepath)
world.query('name == "Bishkek"')

In [ ]:
# Create the base map with center in Bishkek and scrolling limits
m = folium.Map(location=[42.87503,74.58326],
                max_bounds=True,
                min_lat= 42.7,
                max_lat= 43,
                min_lon= 74.4,
                max_lon= 74.8,
               zoom_start=13)

# Add a second map style option for the initial map view
folium.TileLayer('Cartodb Positron').add_to(m)
MousePosition().add_to(m)

# Add Bishkek Administrative Boundaries as a separate group for layers control 
Boundaries = FeatureGroup("Bishkek Administrative").add_to(m)
kgz = gpd.read_file('/kaggle/input/2023-11-10-bishkek-air-condition/kgz_administrative.geojson')

Choropleth(
    geo_data=kgz.__geo_interface__,
    name="boundary",
    fill_color="#99A8B1",
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Administrative boundaries of Bishkek",
).add_to(Boundaries)


# Add Sensors as a group for the layers control
Sensors = FeatureGroup("Sensors").add_to(m)

def color_producer_PM10 (val):
    if val <= 40:
        return 'forestgreen'
    elif val <= 80:
        return 'yellow'
    elif val <= 120:
        return 'orange'    
    else:
        return 'darkred'

# Clusters and Circles in one feature
mc = MarkerCluster().add_to(Sensors)
for idx, row in sensors.iterrows():
    if not math.isnan(row['lon']) and not math.isnan(row['lat']):
        mc.add_child(
                    Circle(
                        location = [row['lat'], row['lon']],
                        radius=1000,
                        color = color_producer_PM10(row['P1']),
                        fill_opacity=0.7,
                        fill_color=color_producer_PM10(row['P1']),
                        tooltip=f"PM10 = {int(row.P1)}, PM2.5 = {int(row.P2)}"
                    )
        )

# Add layers control
folium.LayerControl().add_to(m)
        
# Display the map
m